In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2, f_regression

In [2]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
sel = VarianceThreshold(threshold=700)

In [3]:
df_macro = pd.read_csv("macro.csv")
df_train = pd.read_csv("train.csv").drop(columns=["id"])
df_macro['timestamp'] = pd.to_datetime(df_macro['timestamp'])
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
y = df_train[["price_doc","timestamp"]]
df_M = pd.merge_asof(y,df_macro,on="timestamp",direction="forward")
y_m = df_M["price_doc"]
yf = df_train["price_doc"]

In [4]:
Xm = df_M.drop(columns=['timestamp','price_doc'])._get_numeric_data().to_numpy()
imp_mean.fit(Xm)
Mx = imp_mean.transform(Xm)

ym = y_m.to_numpy()

X_top = SelectKBest(f_regression, k=10).fit_transform(Mx, ym)

top_df = pd.DataFrame(data=X_top,columns=["col_1","col_2","col_3","col_4","col_5","col_6","col_7","col_8","col_9","col_10"])

top_cols = top_df.columns
M_cols = df_M.columns
final_cols = ["timestamp"]
for col in M_cols:
    for t_col in top_cols:
        tf = np.where(df_M[col] == top_df[t_col],0,1)
        if sum(tf) == 0:
            final_cols.append(col)
            break
df_finalM = df_M[final_cols]

C:\Users\Mason\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:302: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\Mason\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:307: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\Mason\AppData\Local\Programs\Python\Python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\Mason\AppData\Local\Programs\Python\Python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\Mason\AppData\Local\Programs\Python\Python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered

In [5]:
df = pd.merge_asof(df_finalM,df_train,on="timestamp",direction="forward")

In [6]:
yf = df["price_doc"]
df_F = df.drop(['timestamp', 'price_doc'], axis=1)

In [7]:
Xf = df_F._get_numeric_data().to_numpy()
imp_mean.fit(Xf)
X = imp_mean.transform(Xf)
y = yf.to_numpy()

Xf_top = SelectKBest(f_regression, k=50).fit_transform(X, y)

In [8]:
Xf_top

array([[43.        , 27.        ,  1.95747857, ...,  9.        ,
        52.        ,  4.        ],
       [34.        , 19.        ,  1.95747857, ..., 15.        ,
        66.        , 14.        ],
       [43.        , 29.        ,  1.95747857, ..., 10.        ,
        67.        , 10.        ],
       ...,
       [36.        , 19.        ,  1.        , ..., 41.        ,
        80.        ,  6.        ],
       [36.        , 19.        ,  1.        , ..., 41.        ,
        80.        ,  6.        ],
       [36.        , 19.        ,  1.        , ..., 41.        ,
        80.        ,  6.        ]])

In [9]:
fin_cols = []
for i in range(50):
    fin_cols.append("col_" + str(i+1))
                    
fin_df = pd.DataFrame(data=Xf_top,columns=fin_cols)

In [10]:
final_features = ["timestamp"]
df_cols = df.columns
for f_col in fin_cols:
    for df_col in df_cols:
        tf = np.where(fin_df[f_col] == df[df_col],0,1)
        if sum(tf) == 0:
            final_features.append(df_col)
            break

In [11]:
df[final_features].to_csv('josiah_final_data.csv', index=False)

,timestamp,full_sq,sport_objects_raion,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,park_km,ttk_km,sadovoe_km,...,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,sport_count_5000,market_count_5000
0,2011-08-20,43,7,2.590241,1.131260,13.575119,1.131260,2.158587,10.918587,13.100618,...,52,4036616,152,12,39,48,40,9,52,4
1,2011-08-23,34,6,0.936700,0.647337,7.620630,0.635053,0.550690,3.103996,6.444333,...,40,2034942,177,9,49,65,36,15,66,14
2,2011-08-27,43,5,2.120999,1.637996,17.351515,1.445960,0.374848,2.927487,6.963403,...,35,1572990,122,10,29,45,25,10,67,10
3,2011-09-01,89,17,1.489049,0.984537,11.565624,0.963802,0.078090,14.606501,17.457198,...,22,942180,61,4,7,21,15,11,26,3
4,2011-09-05,77,25,1.257186,0.876620,8.266305,0.688859,0.258289,1.721834,0.046810,...,114,3503058,2283,143,566,578,552,319,195,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,2015-06-30,36,11,3.000099,1.596772,17.619417,1.468285,1.198874,5.886573,8.641846,...,47,1145385,359,15,77,119,102,41,80,6
30467,2015-06-30,36,11,3.000099,1.596772,17.619417,1.468285,1.198874,5.886573,8.641846,...,47,1145385,359,15,77,119,102,41,80,6
30468,2015-06-30,36,11,3.000099,1.596772,17.619417,1.468285,1.198874,5.886573,8.641846,...,47,1145385,359,15,77,119,102,41,80,6
30469,2015-06-30,36,11,3.000099,1.596772,17.619417,1.468285,1.198874,5.886573,8.641846,...,47,1145385,359,15,77,119,102,41,80,6
